In [2]:
# Guillermo Tafoya Milo < A01633790 >
# Emilio Yoltic Martinez Gutierrez < A01540484 >
# Alejandro Mauricio Maqueo Huerta < A01620649 >
# Narhari Olalde < A01284077 >



# Install mesa for google colab
!pip install mesa

"pip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [3]:
# 'Model' sirve para definir los atributos a nivel del modelo, maneja los agentes
# 'Agent' es la unidad atómica y puede ser contenido en múltiples instancias en los modelos
from mesa import Agent, Model 

# 'SingleGrid' sirve para forzar a un solo objeto por celda (nuestro objetivo en este "juego")
from mesa.space import MultiGrid

# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import SimultaneousActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación), útil para visualizar
from mesa.datacollection import DataCollector

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

import random

In [4]:
# Tamaño del espacio
m = 10
n = 10

# Número de agentes

num_agentes = 3

# Porcentaje de celdas normalmente sucias

p_sucias = 0.6

# Tiempo máximo de ejecución del algoritmo

tiempo_max = 2

# Nivel de carga máximo de los agentes. Puede ser en porcentaje o en unidades de carga.

carga_max = 100

In [5]:
def get_grid(model):
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for obj in cell_content:
            if isinstance(obj, CleanAgent):
                grid[x][y] = 2
            elif isinstance(obj, Cell):
                grid[x][y] = obj.state
    return grid
class CleanAgent(Agent):
    def __init__(self,unique_id,model):
        super().__init__(unique_id, model)
        self.next_state = None
        self.moves = 0
        #self.bateria = bateria
    def advance(self):
        """
        Define el nuevo estado calculado del método step.
        """
        neighbours = self.model.grid.get_neighbors(
        self.pos, 
        moore = True,
        include_center = True)
        for neighbour in neighbours:
            if isinstance(neighbour, Cell) and self.pos == neighbour.pos:
                neighbour.state = neighbour.next_state
                break
        self.model.grid.move_agent(self, self.next_state)
    def step(self):
        neighbours = self.model.grid.get_neighbors(
            self.pos,
            moore=True,
            include_center=False)
        for neighbour in neighbours:
            if isinstance(neighbour, Cell)  and self.pos == neighbour.pos:
                if neighbour.state:
                    neighbour.next_state = 0
                    self.next_state = self.pos
            else:
                self.moves += 1
                neighbours_2 = self.model.grid.get_neighborhood(
                self.pos, 
                moore = True,
                include_center = False)
                neighbour.next_state = 0
                ### Implement shortest path ###
                self.next_state = self.make_decision(neighbours_2)
            break
    def make_decision(self,neighbours):
        """
        Define la decisión del agente.
        """
        return random.choice(neighbours) # Algoritmo de decisión aleatorio.
class Cell(Agent):
    def __init__(self,unique_id,model,state):
        super().__init__(unique_id, model)
        self.next_state = None
        self.pos = unique_id
        self.state = state
class Board(Model):
    def __init__(self,m,n,num_agentes,p_sucias):
        self.num_agentes = num_agentes
        self.p_sucias = p_sucias
        self.schedule = SimultaneousActivation(self)
        self.grid = MultiGrid(m,n,False) # Bool defines toroidal boundary
        num_celdas_sucias = int(m*n*p_sucias)
        for (content,x,y) in self.grid.coord_iter():
            num = random.randint(0,1)
            if num == 1 and num_celdas_sucias > 0:
                a = Cell((x,y), self, 1)
                num_celdas_sucias -= 1
            else:
                a = Cell((x,y), self, 0)
            self.grid.place_agent(a,(x,y))
            self.schedule.add(a)
        for id in range(num_agentes):
            r = CleanAgent(id, self)
            self.grid.place_agent(r, (1,1))
            self.schedule.add(r)

        self.datacollector = DataCollector(
            model_reporters = {"Grid":get_grid})

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

    def allClean(self):
        for (content, x, y) in self.grid.coord_iter():
            for obj in content:
                if isinstance(obj, Cell) and obj.state == 1:
                    return False
        return True

    def numDirty(self):
        cantidad_celdas_sucias = 0
        for (content, x, y) in self.grid.coord_iter():
            for obj in content:
                if isinstance(obj, Cell) and obj.state == 1:
                    cantidad_celdas_sucias = cantidad_celdas_sucias + 1
        return cantidad_celdas_sucias

    def count_moves(self, M, N, num_agentes):
        cont = 0
        for i in range (N*M, N*M + num_agentes):
            cont = cont + self.schedule.agents[i].moves
        return cont

In [6]:
model = Board(m, n, num_agentes, p_sucias)
start_time = time.time()
num_mov = 0
while((time.time() - start_time) < tiempo_max and not model.allClean()):
    model.step()
num_mov = model.count_moves(m, n, num_agentes)

In [7]:
# Reporte
print('Tiempo: ', str(datetime.timedelta(seconds=(time.time() - start_time))))
print('Porcentaje de celdas sucias: ', str(model.numDirty()/(m*n)*100)+"%")
print('Movimientos totales: ', str(num_mov))
print('Movimientos por agente: ', str(num_mov/num_agentes))

Tiempo:  0:00:00.095744
Porcentaje de celdas sucias:  0.0%
Movimientos totales:  840
Movimientos por agente:  280.0


In [8]:
%%capture

all_grid = model.datacollector.get_model_vars_dataframe()
fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [9]:
print(all_grid.iloc[0][0])

[[1. 0. 0. 0. 0. 1. 1. 1. 1. 1.]
 [0. 2. 0. 1. 0. 1. 1. 0. 0. 0.]
 [1. 0. 0. 1. 1. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 0. 1. 0. 0.]
 [1. 0. 1. 1. 1. 0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 1. 1. 0.]]


In [10]:
anim

## Diagramas
### Board
1. Se genera un tablero.
### Celda
1. Se generan celdas sucias al azar.
2. La celda tiene el estado de limpio o sucio como booleano.
### Agente Limpiador
1. Agentes aparecen en posiciones al azar.
2. El agente limpiador se mueve a una celda vecina al azar si la celda actual se encuentra vacia.
3. Si la celda esta sucia, el agente lo limpia y permanece en la misma posición.